# Module 8: Scheduled Tasks

## Welcome to Module 8!

In Module 7, you mastered task automation fundamentals. Now it's time to make your automation **run automatically** using Windows Task Scheduler.

### What You'll Learn

- Understanding Task Scheduler
- Viewing existing scheduled tasks
- Creating scheduled tasks with PowerShell
- Time-based triggers (daily, weekly, at logon)
- Task actions (running scripts and programs)
- Managing tasks (enable, disable, remove)
- Task conditions and settings
- Building automated backup systems

### Why Scheduled Tasks Matter

Scheduled tasks let you:
- **Automate** repetitive maintenance
- **Run** scripts without manual intervention
- **Schedule** backups and reports
- **Monitor** systems 24/7
- **Free** yourself from routine tasks

Let's automate your automation!

## Setup: Prepare Practice Environment

In [1]:
import subprocess
from pathlib import Path
import time

# Practice folder
practice_folder = Path.home() / "Documents" / "AutomationPractice"
practice_folder.mkdir(exist_ok=True)

# Create Scheduler practice subfolder
scheduler_practice = practice_folder / "Scheduler_Practice"
scheduler_practice.mkdir(exist_ok=True)

print(f"Practice folder ready: {scheduler_practice}")
print("Let's schedule some automation!\n")

# Helper function
def run_ps(command, cwd=None):
    """Run a PowerShell command and return output."""
    if cwd is None:
        cwd = str(scheduler_practice)
    
    result = subprocess.run(
        ['powershell', '-Command', command],
        cwd=cwd,
        capture_output=True,
        text=True,
        timeout=30
    )
    
    return result.stdout + result.stderr

print("✓ Helper function ready!")

Practice folder ready: C:\Users\USER\Documents\AutomationPractice\Scheduler_Practice
Let's schedule some automation!

✓ Helper function ready!


## 1. Understanding Task Scheduler

Windows Task Scheduler is a powerful service that runs tasks automatically based on triggers.

### Viewing Existing Scheduled Tasks

In [2]:
output = run_ps('''
Write-Host "=== Viewing Scheduled Tasks ==="
Write-Host ""

# Get all scheduled tasks (limit output for readability)
$tasks = Get-ScheduledTask | Where-Object {$_.State -ne 'Disabled'} | Select-Object -First 10

Write-Host "First 10 Active Scheduled Tasks:"
Write-Host "-" * 60

foreach ($task in $tasks) {
    Write-Host "Task: $($task.TaskName)"
    Write-Host "  Path: $($task.TaskPath)"
    Write-Host "  State: $($task.State)"
    Write-Host ""
}

Write-Host "Total tasks on system: $(Get-ScheduledTask).Count)"
''')

print(output)

=== Viewing Scheduled Tasks ===

First 10 Active Scheduled Tasks:
- * 60
Task: BraveSoftwareUpdateTaskUserS-1-5-21-4115426646-996912538-857568410-1001Core{61401EE7-B245-46F4-8486-6563639C5E60}
  Path: \
  State: Ready

Task: BraveSoftwareUpdateTaskUserS-1-5-21-4115426646-996912538-857568410-1001UA{5FF39A0E-CC07-4054-8004-DE92B82E54B9}
  Path: \
  State: Ready

Task: NVIDIA app SelfUpdate_{B2FE1952-0186-46C3-BAEC-A80AA35AC5B8}
  Path: \
  State: Ready

Task: OneDrive Reporting Task-S-1-5-21-4115426646-996912538-857568410-1001
  Path: \
  State: Ready

Task: OneDrive Standalone Update Task-S-1-5-21-4115426646-996912538-857568410-1001
  Path: \
  State: Ready

Task: OneDrive Startup Task-S-1-5-21-4115426646-996912538-857568410-1001
  Path: \
  State: Ready

Task: RtkAudUService64_BG
  Path: \
  State: Running

Task: ZoomUpdateTaskUser-S-1-5-21-4115426646-996912538-857568410-1001
  Path: \
  State: Ready

Task: AcPowerNotification
  Path: \ASUS\
  State: Running

Task: ArmourySocketServer


### Task Scheduler Components

A scheduled task has three main components:

1. **Trigger**: When the task runs (time, event, logon, etc.)
2. **Action**: What the task does (run program, script, etc.)
3. **Settings**: Additional conditions (idle time, power, network, etc.)

## 2. Creating Basic Scheduled Tasks

### Simple Task: Run Once

In [3]:
output = run_ps('''
Write-Host "=== Creating a Simple Scheduled Task ==="
Write-Host ""

# Create a simple script to run
$scriptContent = @"
Write-Host "Hello from scheduled task!"
Get-Date | Out-File "task_executed.txt"
"@

$scriptPath = Join-Path (Get-Location) "test_script.ps1"
$scriptContent | Out-File -FilePath $scriptPath -Encoding UTF8

Write-Host "Created test script: $scriptPath"
Write-Host ""

# Create action (what to run)
$action = New-ScheduledTaskAction -Execute "PowerShell.exe" -Argument "-NoProfile -ExecutionPolicy Bypass -File `"$scriptPath`""

# Create trigger (when to run) - 2 minutes from now
$triggerTime = (Get-Date).AddMinutes(2)
$trigger = New-ScheduledTaskTrigger -Once -At $triggerTime

Write-Host "Task will run at: $triggerTime"
Write-Host ""

# Register the task
$taskName = "TestTask_Simple"
try {
    # Remove if exists
    Unregister-ScheduledTask -TaskName $taskName -Confirm:$false -ErrorAction SilentlyContinue
    
    # Register new task
    Register-ScheduledTask -TaskName $taskName -Action $action -Trigger $trigger -Description "Simple test task" | Out-Null
    
    Write-Host "Task created successfully: $taskName" -ForegroundColor Green
    
    # View the task
    $task = Get-ScheduledTask -TaskName $taskName
    Write-Host "State: $($task.State)"
    Write-Host "Next Run: $((Get-ScheduledTaskInfo -TaskName $taskName).NextRunTime)"
}
catch {
    Write-Host "Error: $($_.Exception.Message)" -ForegroundColor Red
    Write-Host "Note: Creating scheduled tasks may require administrator privileges"
}
''')

print(output)

=== Creating a Simple Scheduled Task ===

Created test script: C:\Users\USER\Documents\AutomationPractice\Scheduler_Practice\test_script.ps1

Task will run at: 11/14/2025 23:45:04

Task created successfully: TestTask_Simple
State: Ready
Next Run: 11/14/2025 23:45:04



### Daily Task: Run Every Day

In [4]:
output = run_ps('''
Write-Host "=== Creating a Daily Scheduled Task ==="
Write-Host ""

# Create daily script
$dailyScript = @"
# Daily maintenance script
`$logFile = Join-Path (Get-Location) "daily_log.txt"
`$timestamp = Get-Date -Format "yyyy-MM-dd HH:mm:ss"
"[`$timestamp] Daily task executed" | Add-Content `$logFile
Write-Host "Daily task completed at `$timestamp"
"@

$scriptPath = Join-Path (Get-Location) "daily_script.ps1"
$dailyScript | Out-File -FilePath $scriptPath -Encoding UTF8

Write-Host "Created daily script: $scriptPath"
Write-Host ""

# Create action
$action = New-ScheduledTaskAction -Execute "PowerShell.exe" -Argument "-NoProfile -ExecutionPolicy Bypass -File `"$scriptPath`""

# Create daily trigger at 9:00 AM
$trigger = New-ScheduledTaskTrigger -Daily -At "9:00AM"

Write-Host "Task will run daily at 9:00 AM"
Write-Host ""

# Register the task
$taskName = "TestTask_Daily"
try {
    Unregister-ScheduledTask -TaskName $taskName -Confirm:$false -ErrorAction SilentlyContinue
    
    Register-ScheduledTask -TaskName $taskName -Action $action -Trigger $trigger -Description "Daily maintenance task" | Out-Null
    
    Write-Host "Daily task created: $taskName" -ForegroundColor Green
    
    $task = Get-ScheduledTask -TaskName $taskName
    Write-Host "State: $($task.State)"
    Write-Host "Trigger: Daily at 9:00 AM"
}
catch {
    Write-Host "Error: $($_.Exception.Message)" -ForegroundColor Red
    Write-Host "Note: May require administrator privileges"
}
''')

print(output)

=== Creating a Daily Scheduled Task ===

Created daily script: C:\Users\USER\Documents\AutomationPractice\Scheduler_Practice\daily_script.ps1

Task will run daily at 9:00 AM

Daily task created: TestTask_Daily
State: Ready
Trigger: Daily at 9:00 AM



### Weekly Task: Run on Specific Days

In [5]:
output = run_ps('''
Write-Host "=== Creating a Weekly Scheduled Task ==="
Write-Host ""

# Create weekly script
$weeklyScript = @"
# Weekly backup script
`$logFile = Join-Path (Get-Location) "weekly_log.txt"
`$timestamp = Get-Date -Format "yyyy-MM-dd HH:mm:ss"
"[`$timestamp] Weekly backup executed" | Add-Content `$logFile
Write-Host "Weekly backup completed"
"@

$scriptPath = Join-Path (Get-Location) "weekly_script.ps1"
$weeklyScript | Out-File -FilePath $scriptPath -Encoding UTF8

Write-Host "Created weekly script: $scriptPath"
Write-Host ""

# Create action
$action = New-ScheduledTaskAction -Execute "PowerShell.exe" -Argument "-NoProfile -ExecutionPolicy Bypass -File `"$scriptPath`""

# Create weekly trigger - Every Monday and Friday at 6:00 PM
$trigger = New-ScheduledTaskTrigger -Weekly -DaysOfWeek Monday,Friday -At "6:00PM"

Write-Host "Task will run every Monday and Friday at 6:00 PM"
Write-Host ""

# Register the task
$taskName = "TestTask_Weekly"
try {
    Unregister-ScheduledTask -TaskName $taskName -Confirm:$false -ErrorAction SilentlyContinue
    
    Register-ScheduledTask -TaskName $taskName -Action $action -Trigger $trigger -Description "Weekly backup task" | Out-Null
    
    Write-Host "Weekly task created: $taskName" -ForegroundColor Green
    
    $task = Get-ScheduledTask -TaskName $taskName
    Write-Host "State: $($task.State)"
    Write-Host "Trigger: Monday, Friday at 6:00 PM"
}
catch {
    Write-Host "Error: $($_.Exception.Message)" -ForegroundColor Red
}
''')

print(output)

=== Creating a Weekly Scheduled Task ===

Created weekly script: C:\Users\USER\Documents\AutomationPractice\Scheduler_Practice\weekly_script.ps1

Task will run every Monday and Friday at 6:00 PM

Weekly task created: TestTask_Weekly
State: Ready
Trigger: Monday, Friday at 6:00 PM



### At Logon Task: Run When User Logs In

In [6]:
output = run_ps('''
Write-Host "=== Creating an At Logon Task ==="
Write-Host ""

# Create logon script
$logonScript = @"
# Logon initialization script
`$logFile = Join-Path (Get-Location) "logon_log.txt"
`$timestamp = Get-Date -Format "yyyy-MM-dd HH:mm:ss"
"[`$timestamp] User logged in" | Add-Content `$logFile
Write-Host "Logon script executed"
"@

$scriptPath = Join-Path (Get-Location) "logon_script.ps1"
$logonScript | Out-File -FilePath $scriptPath -Encoding UTF8

Write-Host "Created logon script: $scriptPath"
Write-Host ""

# Create action
$action = New-ScheduledTaskAction -Execute "PowerShell.exe" -Argument "-NoProfile -ExecutionPolicy Bypass -File `"$scriptPath`""

# Create logon trigger
$trigger = New-ScheduledTaskTrigger -AtLogOn

Write-Host "Task will run at every user logon"
Write-Host ""

# Register the task
$taskName = "TestTask_Logon"
try {
    Unregister-ScheduledTask -TaskName $taskName -Confirm:$false -ErrorAction SilentlyContinue
    
    Register-ScheduledTask -TaskName $taskName -Action $action -Trigger $trigger -Description "Logon initialization task" | Out-Null
    
    Write-Host "Logon task created: $taskName" -ForegroundColor Green
    
    $task = Get-ScheduledTask -TaskName $taskName
    Write-Host "State: $($task.State)"
    Write-Host "Trigger: At user logon"
}
catch {
    Write-Host "Error: $($_.Exception.Message)" -ForegroundColor Red
}
''')

print(output)

=== Creating an At Logon Task ===

Created logon script: C:\Users\USER\Documents\AutomationPractice\Scheduler_Practice\logon_script.ps1

Task will run at every user logon

Logon task created: TestTask_Logon
State: 
Trigger: At user logon
Register-ScheduledTask : Access is denied.
At line:34 char:5
+     Register-ScheduledTask -TaskName $taskName -Action $action -Trigg ...
+     ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : PermissionDenied: (PS_ScheduledTask:Root/Microsoft/...S_ScheduledTask) [Register-Schedul 
   edTask], CimException
    + FullyQualifiedErrorId : HRESULT 0x80070005,Register-ScheduledTask
 
Get-ScheduledTask : No MSFT_ScheduledTask objects found with property 'TaskName' equal to 'TestTask_Logon'.  Verify 
the value of the property and retry.
At line:38 char:13
+     $task = Get-ScheduledTask -TaskName $taskName
+             ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : ObjectNotFound: (TestTask_L

## 3. Managing Scheduled Tasks

### Viewing Task Information

In [7]:
output = run_ps('''
Write-Host "=== Viewing Task Information ==="
Write-Host ""

# Get our test tasks
$taskNames = @("TestTask_Simple", "TestTask_Daily", "TestTask_Weekly", "TestTask_Logon")

foreach ($taskName in $taskNames) {
    try {
        $task = Get-ScheduledTask -TaskName $taskName -ErrorAction Stop
        $info = Get-ScheduledTaskInfo -TaskName $taskName
        
        Write-Host "Task: $taskName"
        Write-Host "  State: $($task.State)"
        Write-Host "  Description: $($task.Description)"
        Write-Host "  Last Run: $($info.LastRunTime)"
        Write-Host "  Next Run: $($info.NextRunTime)"
        Write-Host "  Last Result: $($info.LastTaskResult)"
        Write-Host ""
    }
    catch {
        Write-Host "Task not found: $taskName"
        Write-Host ""
    }
}
''')

print(output)

=== Viewing Task Information ===

Task: TestTask_Simple
  State: Ready
  Description: Simple test task
  Last Run: 11/30/1999 00:00:00
  Next Run: 11/14/2025 23:45:04
  Last Result: 267011

Task: TestTask_Daily
  State: Ready
  Description: Daily maintenance task
  Last Run: 11/30/1999 00:00:00
  Next Run: 11/15/2025 09:00:00
  Last Result: 267011

Task: TestTask_Weekly
  State: Ready
  Description: Weekly backup task
  Last Run: 11/30/1999 00:00:00
  Next Run: 11/17/2025 18:00:00
  Last Result: 267011

Task not found: TestTask_Logon




### Running Tasks Manually

In [8]:
output = run_ps('''
Write-Host "=== Running Task Manually ==="
Write-Host ""

try {
    $taskName = "TestTask_Daily"
    
    Write-Host "Running task: $taskName"
    Start-ScheduledTask -TaskName $taskName
    
    # Wait a moment for task to start
    Start-Sleep -Seconds 2
    
    # Check if log file was created
    $logFile = Join-Path (Get-Location) "daily_log.txt"
    if (Test-Path $logFile) {
        Write-Host "Task executed successfully!" -ForegroundColor Green
        Write-Host "Log contents:"
        Get-Content $logFile | Select-Object -Last 3
    }
    else {
        Write-Host "Task started (log file may appear shortly)"
    }
}
catch {
    Write-Host "Error: $($_.Exception.Message)" -ForegroundColor Red
}
''')

print(output)

=== Running Task Manually ===

Running task: TestTask_Daily
Task started (log file may appear shortly)



### Enabling and Disabling Tasks

In [9]:
output = run_ps('''
Write-Host "=== Enabling and Disabling Tasks ==="
Write-Host ""

try {
    $taskName = "TestTask_Weekly"
    
    # Disable the task
    Write-Host "Disabling task: $taskName"
    Disable-ScheduledTask -TaskName $taskName | Out-Null
    
    $task = Get-ScheduledTask -TaskName $taskName
    Write-Host "  State after disable: $($task.State)" -ForegroundColor Yellow
    Write-Host ""
    
    # Enable the task
    Write-Host "Enabling task: $taskName"
    Enable-ScheduledTask -TaskName $taskName | Out-Null
    
    $task = Get-ScheduledTask -TaskName $taskName
    Write-Host "  State after enable: $($task.State)" -ForegroundColor Green
}
catch {
    Write-Host "Error: $($_.Exception.Message)" -ForegroundColor Red
}
''')

print(output)

=== Enabling and Disabling Tasks ===

Disabling task: TestTask_Weekly
  State after disable: Disabled

Enabling task: TestTask_Weekly
  State after enable: Ready



### Removing Tasks

In [10]:
output = run_ps('''
Write-Host "=== Removing Scheduled Tasks ==="
Write-Host ""

# Remove simple task (we only need it once)
try {
    $taskName = "TestTask_Simple"
    
    # Check if exists
    $task = Get-ScheduledTask -TaskName $taskName -ErrorAction SilentlyContinue
    
    if ($task) {
        Write-Host "Removing task: $taskName"
        Unregister-ScheduledTask -TaskName $taskName -Confirm:$false
        Write-Host "  Task removed successfully" -ForegroundColor Green
    }
    else {
        Write-Host "Task not found: $taskName"
    }
}
catch {
    Write-Host "Error: $($_.Exception.Message)" -ForegroundColor Red
}

Write-Host ""
Write-Host "Remaining test tasks:"
$remainingTasks = @("TestTask_Daily", "TestTask_Weekly", "TestTask_Logon")
foreach ($taskName in $remainingTasks) {
    $exists = Get-ScheduledTask -TaskName $taskName -ErrorAction SilentlyContinue
    if ($exists) {
        Write-Host "  - $taskName (State: $($exists.State))"
    }
}
''')

print(output)

=== Removing Scheduled Tasks ===

Removing task: TestTask_Simple
  Task removed successfully

Remaining test tasks:
  - TestTask_Daily (State: Ready)
  - TestTask_Weekly (State: Ready)



## 4. Advanced Task Configuration

### Task with Settings and Conditions

In [11]:
output = run_ps('''
Write-Host "=== Advanced Task Configuration ==="
Write-Host ""

# Create advanced script
$advancedScript = @"
`$logFile = Join-Path (Get-Location) "advanced_log.txt"
`$timestamp = Get-Date -Format "yyyy-MM-dd HH:mm:ss"
"[`$timestamp] Advanced task executed" | Add-Content `$logFile
"@

$scriptPath = Join-Path (Get-Location) "advanced_script.ps1"
$advancedScript | Out-File -FilePath $scriptPath -Encoding UTF8

# Create action
$action = New-ScheduledTaskAction -Execute "PowerShell.exe" -Argument "-NoProfile -ExecutionPolicy Bypass -File `"$scriptPath`""

# Create trigger (daily at 10:00 AM)
$trigger = New-ScheduledTaskTrigger -Daily -At "10:00AM"

# Create settings
$settings = New-ScheduledTaskSettingsSet `
    -AllowStartIfOnBatteries `
    -DontStopIfGoingOnBatteries `
    -StartWhenAvailable `
    -RunOnlyIfNetworkAvailable `
    -ExecutionTimeLimit (New-TimeSpan -Minutes 10)

Write-Host "Task Settings Configured:"
Write-Host "  - Allow start if on batteries"
Write-Host "  - Don't stop if going on batteries"
Write-Host "  - Start when available (if missed)"
Write-Host "  - Run only if network available"
Write-Host "  - Execution time limit: 10 minutes"
Write-Host ""

# Register the task
$taskName = "TestTask_Advanced"
try {
    Unregister-ScheduledTask -TaskName $taskName -Confirm:$false -ErrorAction SilentlyContinue
    
    Register-ScheduledTask -TaskName $taskName -Action $action -Trigger $trigger -Settings $settings -Description "Advanced task with settings" | Out-Null
    
    Write-Host "Advanced task created: $taskName" -ForegroundColor Green
}
catch {
    Write-Host "Error: $($_.Exception.Message)" -ForegroundColor Red
}
''')

print(output)

=== Advanced Task Configuration ===

Task Settings Configured:
  - Allow start if on batteries
  - Don't stop if going on batteries
  - Start when available (if missed)
  - Run only if network available
  - Execution time limit: 10 minutes

Advanced task created: TestTask_Advanced



### Task with Multiple Triggers

In [12]:
output = run_ps('''
Write-Host "=== Task with Multiple Triggers ==="
Write-Host ""

# Create multi-trigger script
$multiScript = @"
`$logFile = Join-Path (Get-Location) "multi_trigger_log.txt"
`$timestamp = Get-Date -Format "yyyy-MM-dd HH:mm:ss"
"[`$timestamp] Multi-trigger task executed" | Add-Content `$logFile
"@

$scriptPath = Join-Path (Get-Location) "multi_trigger_script.ps1"
$multiScript | Out-File -FilePath $scriptPath -Encoding UTF8

# Create action
$action = New-ScheduledTaskAction -Execute "PowerShell.exe" -Argument "-NoProfile -ExecutionPolicy Bypass -File `"$scriptPath`""

# Create multiple triggers
$trigger1 = New-ScheduledTaskTrigger -Daily -At "8:00AM"
$trigger2 = New-ScheduledTaskTrigger -AtLogOn
$triggers = @($trigger1, $trigger2)

Write-Host "Configured Triggers:"
Write-Host "  1. Daily at 8:00 AM"
Write-Host "  2. At user logon"
Write-Host ""

# Register the task
$taskName = "TestTask_MultiTrigger"
try {
    Unregister-ScheduledTask -TaskName $taskName -Confirm:$false -ErrorAction SilentlyContinue
    
    Register-ScheduledTask -TaskName $taskName -Action $action -Trigger $triggers -Description "Task with multiple triggers" | Out-Null
    
    Write-Host "Multi-trigger task created: $taskName" -ForegroundColor Green
    Write-Host "This task will run both daily and at logon"
}
catch {
    Write-Host "Error: $($_.Exception.Message)" -ForegroundColor Red
}
''')

print(output)

=== Task with Multiple Triggers ===

Configured Triggers:
  1. Daily at 8:00 AM
  2. At user logon

Multi-trigger task created: TestTask_MultiTrigger
This task will run both daily and at logon
Register-ScheduledTask : Access is denied.
At line:33 char:5
+     Register-ScheduledTask -TaskName $taskName -Action $action -Trigg ...
+     ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : PermissionDenied: (PS_ScheduledTask:Root/Microsoft/...S_ScheduledTask) [Register-Schedul 
   edTask], CimException
    + FullyQualifiedErrorId : HRESULT 0x80070005,Register-ScheduledTask
 



## 5. Practical Project: Automated Backup System

### Create Backup Script

In [13]:
# Create comprehensive backup script
backup_script = '''
# Automated Backup Script
param(
    [string]$SourcePath = "C:\\Users\\$env:USERNAME\\Documents\\Important",
    [string]$BackupRoot = "C:\\Users\\$env:USERNAME\\Documents\\Backups",
    [int]$RetentionDays = 7
)

$logFile = Join-Path $BackupRoot "backup_log.txt"

function Write-Log {
    param($Message)
    $timestamp = Get-Date -Format "yyyy-MM-dd HH:mm:ss"
    $logMessage = "[$timestamp] $Message"
    Write-Host $logMessage
    $logMessage | Add-Content $logFile
}

try {
    Write-Log "=== Backup Started ==="
    
    # Create backup root if needed
    if (-not (Test-Path $BackupRoot)) {
        New-Item -Path $BackupRoot -ItemType Directory | Out-Null
        Write-Log "Created backup directory: $BackupRoot"
    }
    
    # Check source exists
    if (-not (Test-Path $SourcePath)) {
        Write-Log "WARNING: Source path not found: $SourcePath"
        Write-Log "Creating test source directory"
        New-Item -Path $SourcePath -ItemType Directory -Force | Out-Null
        "Test file" | Out-File (Join-Path $SourcePath "test.txt")
    }
    
    # Create backup folder with timestamp
    $timestamp = Get-Date -Format "yyyyMMdd_HHmmss"
    $backupFolder = Join-Path $BackupRoot "Backup_$timestamp"
    
    Write-Log "Source: $SourcePath"
    Write-Log "Destination: $backupFolder"
    
    # Copy files
    $files = Get-ChildItem -Path $SourcePath -Recurse -File
    Write-Log "Found $($files.Count) files to backup"
    
    if ($files.Count -gt 0) {
        Copy-Item -Path $SourcePath -Destination $backupFolder -Recurse -Force
        Write-Log "Backup completed successfully"
    }
    else {
        Write-Log "No files to backup"
    }
    
    # Clean old backups
    Write-Log "Cleaning backups older than $RetentionDays days"
    $cutoffDate = (Get-Date).AddDays(-$RetentionDays)
    $oldBackups = Get-ChildItem -Path $BackupRoot -Directory | Where-Object {
        $_.Name -like "Backup_*" -and $_.CreationTime -lt $cutoffDate
    }
    
    foreach ($backup in $oldBackups) {
        Remove-Item -Path $backup.FullName -Recurse -Force
        Write-Log "Removed old backup: $($backup.Name)"
    }
    
    Write-Log "=== Backup Completed Successfully ==="
}
catch {
    Write-Log "ERROR: $($_.Exception.Message)"
    Write-Log "=== Backup Failed ==="
    exit 1
}
'''

(scheduler_practice / "automated_backup.ps1").write_text(backup_script, encoding='utf-8')

print(f"Backup script created: {scheduler_practice / 'automated_backup.ps1'}")
print("\nThis script:")
print("  - Backs up files with timestamp")
print("  - Logs all operations")
print("  - Cleans old backups automatically")
print("  - Handles errors gracefully")

Backup script created: C:\Users\USER\Documents\AutomationPractice\Scheduler_Practice\automated_backup.ps1

This script:
  - Backs up files with timestamp
  - Logs all operations
  - Cleans old backups automatically
  - Handles errors gracefully


### Schedule the Backup Task

In [14]:
output = run_ps(f'''
Write-Host "=== Scheduling Automated Backup ==="
Write-Host ""

$scriptPath = Join-Path (Get-Location) "automated_backup.ps1"

Write-Host "Backup script: $scriptPath"
Write-Host ""

# Create action
$action = New-ScheduledTaskAction -Execute "PowerShell.exe" -Argument "-NoProfile -ExecutionPolicy Bypass -File `"$scriptPath`""

# Create trigger - Daily at 11:00 PM
$trigger = New-ScheduledTaskTrigger -Daily -At "11:00PM"

# Create settings
$settings = New-ScheduledTaskSettingsSet `
    -AllowStartIfOnBatteries `
    -DontStopIfGoingOnBatteries `
    -StartWhenAvailable `
    -ExecutionTimeLimit (New-TimeSpan -Hours 1)

Write-Host "Configuration:"
Write-Host "  Trigger: Daily at 11:00 PM"
Write-Host "  Execution Limit: 1 hour"
Write-Host "  Power: Run on battery if needed"
Write-Host ""

# Register the task
$taskName = "AutomatedBackup_Daily"
try {{
    Unregister-ScheduledTask -TaskName $taskName -Confirm:$false -ErrorAction SilentlyContinue
    
    Register-ScheduledTask -TaskName $taskName -Action $action -Trigger $trigger -Settings $settings -Description "Automated daily backup" | Out-Null
    
    Write-Host "Backup task created successfully!" -ForegroundColor Green
    Write-Host ""
    
    # Show task info
    $task = Get-ScheduledTask -TaskName $taskName
    $info = Get-ScheduledTaskInfo -TaskName $taskName
    
    Write-Host "Task Details:"
    Write-Host "  Name: $($task.TaskName)"
    Write-Host "  State: $($task.State)"
    Write-Host "  Next Run: $($info.NextRunTime)"
    Write-Host ""
    
    Write-Host "Your files will be backed up automatically every day at 11:00 PM!" -ForegroundColor Cyan
}}
catch {{
    Write-Host "Error: $($_.Exception.Message)" -ForegroundColor Red
}}
''')

print(output)

=== Scheduling Automated Backup ===

Backup script: C:\Users\USER\Documents\AutomationPractice\Scheduler_Practice\automated_backup.ps1

Configuration:
  Trigger: Daily at 11:00 PM
  Execution Limit: 1 hour
  Power: Run on battery if needed

Backup task created successfully!

Task Details:
  Name: AutomatedBackup_Daily
  State: Ready
  Next Run: 11/15/2025 23:00:00

Your files will be backed up automatically every day at 11:00 PM!



### Test the Backup Task

In [15]:
output = run_ps('''
Write-Host "=== Testing Backup Task ==="
Write-Host ""

try {
    $taskName = "AutomatedBackup_Daily"
    
    Write-Host "Running backup task manually..."
    Start-ScheduledTask -TaskName $taskName
    
    Write-Host "Waiting for task to complete..."
    Start-Sleep -Seconds 5
    
    # Check log file using Join-Path
    $docsPath = Join-Path $env:USERPROFILE "Documents"
    $backupsPath = Join-Path $docsPath "Backups"
    $logPath = Join-Path $backupsPath "backup_log.txt"
    
    if (Test-Path $logPath) {
        Write-Host "Backup executed successfully!" -ForegroundColor Green
        Write-Host ""
        Write-Host "Log output (last 10 lines):"
        Write-Host "-" * 60
        Get-Content $logPath | Select-Object -Last 10
    }
    else {
        Write-Host "Task is running (log may appear shortly)"
    }
}
catch {
    Write-Host "Error: $($_.Exception.Message)" -ForegroundColor Red
}
''')

print(output)

=== Testing Backup Task ===

Running backup task manually...
Waiting for task to complete...
Backup executed successfully!

Log output (last 10 lines):
- * 60
[2025-11-14 23:41:12] Backup completed successfully
[2025-11-14 23:41:12] Cleaning backups older than 7 days
[2025-11-14 23:41:12] === Backup Completed Successfully ===
[2025-11-14 23:44:08] === Backup Started ===
[2025-11-14 23:44:08] Source: C:\Users\USER\Documents\Important
[2025-11-14 23:44:08] Destination: C:\Users\USER\Documents\Backups\Backup_20251114_234408
[2025-11-14 23:44:08] Found 1 files to backup
[2025-11-14 23:44:08] Backup completed successfully
[2025-11-14 23:44:08] Cleaning backups older than 7 days
[2025-11-14 23:44:08] === Backup Completed Successfully ===



## Summary and Key Takeaways

Congratulations! You've mastered Windows Task Scheduler automation.

### What You Learned:

✓ **Task Scheduler Basics** - Understanding components  
✓ **Viewing Tasks** - Inspecting existing scheduled tasks  
✓ **Creating Tasks** - Simple, daily, weekly, logon triggers  
✓ **Managing Tasks** - Run, enable, disable, remove  
✓ **Advanced Configuration** - Settings, conditions, multiple triggers  
✓ **Automated Backup** - Production-ready backup system  
✓ **Best Practices** - Error handling, logging, retention  

### Quick Reference: Scheduled Tasks

```powershell
# View tasks
Get-ScheduledTask
Get-ScheduledTask -TaskName "MyTask"
Get-ScheduledTaskInfo -TaskName "MyTask"

# Create action
$action = New-ScheduledTaskAction -Execute "PowerShell.exe" -Argument "-File script.ps1"

# Create triggers
$trigger = New-ScheduledTaskTrigger -Once -At (Get-Date)
$trigger = New-ScheduledTaskTrigger -Daily -At "9:00AM"
$trigger = New-ScheduledTaskTrigger -Weekly -DaysOfWeek Monday,Friday -At "6:00PM"
$trigger = New-ScheduledTaskTrigger -AtLogOn

# Create settings
$settings = New-ScheduledTaskSettingsSet -AllowStartIfOnBatteries -DontStopIfGoingOnBatteries

# Register task
Register-ScheduledTask -TaskName "MyTask" -Action $action -Trigger $trigger -Settings $settings

# Manage tasks
Start-ScheduledTask -TaskName "MyTask"
Enable-ScheduledTask -TaskName "MyTask"
Disable-ScheduledTask -TaskName "MyTask"
Unregister-ScheduledTask -TaskName "MyTask" -Confirm:$false
```

### Best Practices

1. **Always use full paths** in scheduled task scripts
2. **Add logging** to track task execution
3. **Set execution time limits** to prevent runaway tasks
4. **Test manually first** before scheduling
5. **Use -ExecutionPolicy Bypass** for script execution
6. **Handle errors gracefully** with Try/Catch
7. **Include cleanup logic** for old data
8. **Document task purpose** in description
9. **Monitor task results** regularly
10. **Use -StartWhenAvailable** for missed runs

### Common Use Cases

- Daily backups
- Log file rotation
- Disk cleanup
- Report generation
- System health checks
- Data synchronization
- Update checks
- Monitoring alerts
- Automated maintenance
- Environment setup at logon

### Next Steps

In **Module 9: System Information**, you'll learn:
- Gathering comprehensive system data
- Monitoring disk usage and health
- Managing processes and services
- Network information and diagnostics
- Creating system health reports

You're now ready to build fully automated workflows!

## Cleanup

Run this cell to remove all practice tasks and files:

In [16]:
import shutil

print("Cleaning up test tasks and files...\n")

# Remove test tasks
output = run_ps('''
$testTasks = @(
    "TestTask_Simple",
    "TestTask_Daily",
    "TestTask_Weekly",
    "TestTask_Logon",
    "TestTask_Advanced",
    "TestTask_MultiTrigger",
    "AutomatedBackup_Daily"
)

Write-Host "Removing test scheduled tasks:"
foreach ($taskName in $testTasks) {
    try {
        $task = Get-ScheduledTask -TaskName $taskName -ErrorAction SilentlyContinue
        if ($task) {
            Unregister-ScheduledTask -TaskName $taskName -Confirm:$false
            Write-Host "  ✓ Removed: $taskName"
        }
    }
    catch {
        # Task doesn't exist or couldn't be removed
    }
}

Write-Host ""
Write-Host "All test tasks removed."
''')

print(output)

# Remove practice folder
if scheduler_practice.exists():
    shutil.rmtree(scheduler_practice)
    print(f"✓ Removed {scheduler_practice}")
    print("\nAll practice files deleted.")
    print("The main AutomationPractice folder remains for future modules.")
else:
    print("Practice folder already cleaned up!")

Cleaning up test tasks and files...



Removing test scheduled tasks:
  û Removed: TestTask_Daily
  û Removed: TestTask_Weekly
  û Removed: TestTask_Advanced
  û Removed: AutomatedBackup_Daily

All test tasks removed.

✓ Removed C:\Users\USER\Documents\AutomationPractice\Scheduler_Practice

All practice files deleted.
The main AutomationPractice folder remains for future modules.
